This Static Model takes the CMAQ data as input and generates .npy downscaled data based on the given resolution which can be edited below. This resolution MUST be a multiple of 356.

In [1]:
import numpy as np
import math
import os.path
from os import path
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.stats import invgamma

%matplotlib inline

global resolution
global w_h
global s
global date

# 356x356 km grid
resolution = 2            # default: 4 km resolution
w_h = int(356/resolution) # default: 89x89 grid
s = int(w_h**2)           # default: 7921 sites
date = '12/30/16'           # default: 1/1/16

data = list(range(s))
coords_toCMAQ = list(range(s))

def progressBar(current, total, barLength):
    percent = float(current) * 100 / total
    arrow   = '-' * int(percent/100 * barLength - 1) + '>'
    spaces  = ' ' * (barLength - len(arrow))

    print('Progress: [%s%s] %d %%' % (arrow, spaces, percent), end='\r')

Estimations below are derived from Berrocal et. al trace plots.
We assume the multiplicative bias of the numerical model output is constant in space.
Thus, in fitting the static downscalter, we set the local adjustment Beta_1(s) to zero.

In [2]:
#overall additive and multiplicative bias of CMAQ
beta_0, beta_1 = 3.2, 0.5

#nugget variance for white noise process
tau_2 = 0.10

#spatial decay parameter for Gaussian process wj(s)
phi_0 = .003

#relevant elements of matrix A
A_11 = 1.1
#this will be the value we predict in our model

First, we define the Gaussian Process for the model. This can take a LONG time since the covariance matrix is s^2/s^2, so we have the ability to load a previous matrix as an .npy file and we always save our new one.

In [3]:
def kernel(phi=-.003):
    """
    Exponential kernel with spatial decay parameter phi.
    Default resolution=4km, i.e. 89x89 grid.
    """
    
    global resolution
    global w_h
    global s
    
    # checks for saved covariance matrix
    if path.exists("covariance_" + str(resolution) + ".npy"):
        with open("covariance_" + str(resolution) + ".npy", 'rb') as f:
            c = np.load(f, allow_pickle=False)
            return c

    # generates covariance matrix
    
    print("Generating coordinates for %d sites..." % (s))
    s_coords = [[0 for i in range(w_h)] for j in range(w_h)] 
    for x in range(w_h):
        for y in range(w_h):
            s_coords[x][y] = [x, y]     
    
    print("Generating %s x %s empty matrix..." % (s, s))
    cov = list(range(s))
    for i in range(s):
        progressBar(i, s, barLength = 80)
        cov[i] = list(range(s))
        
    print("Generating %s x %s covariance matrix..." % (s, s))
    for y in range(s):
        progressBar(y, s, barLength = 80)
        for x in range(s):
            cov[x][y] = np.exp(phi * math.sqrt((s_coords[y%w_h][y//w_h][0] - s_coords[x%w_h][x//w_h][0])**2 + \
                                               (s_coords[y%w_h][y//w_h][1] - s_coords[x%w_h][x//w_h][1])**2))
    
    # saves and returns new covariance matrix
    with open("covariance_" + str(resolution) + ".npy", 'wb') as f:
        np.save(f, cov, allow_pickle=False)
        return cov

Now we can generate a s-sized sample from the multivariate normal. Similar to the covariance matrix, this can take a long time, so we include the ability to load or save.

In [4]:
# checks for saved sample
if path.exists("sample_" + str(resolution) + ".npy"):
    with open("sample_" + str(resolution) + ".npy", 'rb') as f:
        sample = np.load(f, allow_pickle=False)

# otherwise, generates new sample
else:
    with open("sample_" + str(resolution) + ".npy", 'wb') as f:
        mu = np.zeros(s)
        cov = kernel()
        print("Generating sample...")
        sample = np.random.multivariate_normal(mu, cov, 1)
        sample = sample[0]
        np.save(f, sample, allow_pickle=False)

Generating sample...


<ipython-input-4-21a546e71798>:12: RuntimeWarning: covariance is not positive-semidefinite.
  sample = np.random.multivariate_normal(mu, cov, 1)


Now we can start to work with the CMAQ data.

In [5]:
# import and clean dataset
CMAQ_data = pd.read_csv('California_CMAQ_PM2.5_Output_all.csv', \
                         usecols = ['Date','Latitude', 'Longitude', 'PM'])

# extracts data from chosen date
data_start = 0
data_end = 0
while CMAQ_data.Date[data_start] != date:
    data_start += 1
data_end = data_start
while CMAQ_data.Date[data_end] == date:
    data_end += 1
data_start += -1
    
CMAQ_data = CMAQ_data.drop(list(range(data_end+1,673251)))
CMAQ_data = CMAQ_data.drop(list(range(0,data_start)))

# organizes data further
CMAQ_data = {'Lat': CMAQ_data.Latitude,
             'Long': CMAQ_data.Longitude,
             'PM': CMAQ_data.PM}

# saves PM data length (this will be important soon)
PM_data_length = len(CMAQ_data['PM'])

""" FOR TESTING
print(PM_data_length)
print(len(CMAQ_data['Long']))
print(len(CMAQ_data['Lat']))
print(CMAQ_data['Long'][j])
"""

# and even further
CMAQ_coords = list(range(PM_data_length-1))
for a in range(data_start, data_end):
    CMAQ_coords[a-data_start] = [CMAQ_data['Long'][a], CMAQ_data['Lat'][a]]

We need to bind each site to a CMAQ sampling site. First, we must find the coordinates of each site. Then we can find each site's matching CMAQ sampling site. We do this in the slightly terrible method seen below.

In [6]:
# lat: 35.0, 38.2, diff 3.2
# long: -121.7, -117.79, diff 3.91'
for y in range(w_h):
    for x in range(w_h):

        # assign longitudes and latitudes to each site
        coords_toCMAQ[x + w_h*y] = [[-117.79 - x * (0.011189 * resolution), 35 + y * (.008994 * resolution)]]

with open("coords_" + str(resolution) + ".npy", 'wb') as f:
    np.save(f, coords_toCMAQ)# allow_pickle=False)

# locates nearest CMAQ sampling site, replaces coords_toCMAQ lat/long data with PM2.5 data 
# from this nearest sampling site.
for y in range(w_h):
    #progressBar(y, w_h, barLength = 80)
    for x in range(w_h):
        distances = []

        distances = cdist(coords_toCMAQ[x + w_h*y], CMAQ_coords, 'euclidean')
        coords_toCMAQ[x + w_h*y] = CMAQ_data['PM'][np.argmin(distances) + data_start]

Finally, we can put all the pieces together.

In [7]:
# generates noise
noise = np.random.normal(0, 0.10, s)

# final model
for i in range(s):
    data[i] = (beta_0 + A_11 * sample[i] + beta_1 * math.sqrt(coords_toCMAQ[i]) + noise[i])**2

# saves data
with open("data_" + str(resolution) + ".npy", 'wb') as f:
        np.save(f, data, allow_pickle=False)

In [8]:
print(data[0:150])

[20.83730532990872, 20.43970219436185, 21.8036721347045, 23.658443256836524, 22.63382556127829, 23.647989642820296, 21.97681157782389, 22.010476244091283, 22.824253685228577, 22.152377371171998, 19.897204122562083, 23.207734871247997, 21.527182117304694, 22.71203441069201, 22.376178305939476, 21.413070564513912, 21.903721580516695, 22.090906568238342, 21.242233881079827, 22.199741225019796, 21.90021851479127, 20.70541114978971, 21.795910431220126, 21.976016497079605, 20.589931729351104, 23.151002319932957, 22.193168061704746, 20.718937610740934, 23.88671127424383, 22.828313162707893, 24.759470921701308, 23.43116405762786, 23.674910714308396, 24.11305863555218, 22.21207939100477, 23.422243292176432, 23.634175840806982, 22.901367760366238, 22.819229521200686, 25.53769988836379, 25.11868487983551, 22.74602475799805, 21.890405523845196, 28.49568262511149, 28.84087580123013, 27.74093060417328, 28.081558460041922, 29.19444595399085, 28.930071957617994, 28.97473857523039, 28.157210249132408, 